In [ ]:
import numpy as np
import matplotlib.pyplot as plt

LENGTH = 3

In [ ]:
class Environment:
    
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1
        self.o = 1
        self.winner = None
        self.ended = False
        self.num_states = 3**(LENGTH * LENGTH)

    def __str__(self):
        self_str = "x = {}, o = {}\n".format(self.x, self.o)
        self_str += "winner = {}, ended = {}, num_states = {}\n".format(self.winner, self.ended, self.num_states)
        self_str += "state = {}\n".format(self.get_state())
        self_str += self.board_str()
        return self_str
    
    def is_empty(self, i, j):
        return self.board[i, j] == 0
    
    def get_winner(self):
        return self.winner
    
    def get_loser(self):
        if self.winner == self.x:
            return self.o
        elif self.winner == self.o:
            return self.x
        else:
            return None
    
    def reward(self, sym):
        if self.game_over():
            if self.get_winner() == sym:
                return 1
            elif self.get_loser() == sym:
                return -1
            else:
                return 0
        else:
            return 0
    
    def get_state(self):
        # The state is represented as a hash h = the decimal value of a 
        # base 3 integer where each digit is 0 if the square is empty, 
        # 1 if it is x, and 2 if it is 0
        k = 8
        h = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                v = 0
                if self.board[i, j] == 0:
                    v = 0
                elif self.board[i, j] == self.x:
                    v = 1
                elif self.board[i, j] == self.o:
                    v = 2
                h += (3**k) * v
                k -= 1
        return h
    
    def set_board_from_xo_string(self, xo_str):
        if len(xo_str) == LENGTH * LENGTH:
            for i in range(LENGTH):
                for j in range(LENGTH):
                    xo_char = xo_str[j + LENGTH * i]
                    if xo_char == 'x':
                        self.board[i, j] = self.x
                    elif xo_char == 'o':
                        self.board[i, j] = self.o
                    else:
                        self.board[i, j] = 0           
    
    def state_base3(self, s):
        ans = ""
        while s > 0:        
            ans = '{}'.format(s % 3) + ans
            s = s // 3
        return ans.zfill(9)
    
    def state_xo(self, s):
        base3 = self.state_base3(s)
        state_board = np.zeros((LENGTH, LENGTH))
        for i in range(LENGTH):
            for j in range(LENGTH):
                val = base3[j + i * LENGTH]
                if int(val) == 1:
                    state_board[i, j] = self.x
                elif int(val) == 2:
                    state_board[i, j] = self.o
        return state_board
    
    def draw_state(self, state):
        b = self.board.copy()
        self.board = self.state_xo(state)
        self.draw_board()
        self.board = b
    
    def game_over(self, force_recalculate=False):
        if not force_recalculate and self.ended:
            return self.ended
        # Rows
        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
        # Columns
        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:, j].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
        # Diagonals                    
        for player in (self.o, self.x):
            if self.board.trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True
            if np.fliplr(self.board).trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True
        # Check for full board (draw)
        if not np.any((self.board == 0)):
            self.winner = None
            self.ended = True
            return True
        # Game still in progress
        self.winner = None
        return False
        
    def board_str(self):
        ans = ""
        for i in range(LENGTH):
            for j in range(LENGTH):
                pchar = ' '
                if self.board[i, j] == self.x:
                    pchar = 'x'
                elif self.board[i, j] == self.o:
                    pchar = 'o'
                if j < 2:                  
                    ans += ' {} |'.format(pchar)
                else:
                    ans += ' {} \n'.format(pchar)
            if i < 2:
                ans += '---+---+---\n'
            else:
                ans += "\n"
        return ans
    
    def draw_board(self):
        print(self.board_str())

In [ ]:
class Agent:
    
    def __init__(self, _eps=1, _alpha=0.5):
        self.eps = _eps
        self.alpha = _alpha
        self.verbose = False
        self.verbose_update = False
        self.state_history = []
        self.V = None
        self.sym = None
        self.epoch = 0
    
    def update_epsilon(self):
        self.epoch += 1
        self.eps = 1.0 / self.epoch

    def setV(self, V):
        self.V = V
    
    def set_symbol(self, sym):
        self.sym = sym
        
    def set_verbose(self, v):
        if v is True:
            self.verbose = True
        else:
            self.verbose = False
    
    def set_verbose_update(self, v):
        if v is True:
            self.verbose_update = True
        else:
            self.verbose_update = False
    
    def reset_history(self):
        self.state_history = []
        
    def take_action(self, env):
        r = np.random.rand()
        best_state = None
        if r < self.eps:
            if self.verbose:
                if self.sym == env.x:
                    print('x', end='')
                elif self.sym == env.o:
                    print('o', end='')
                print(' is taking a random action - exploring')
            possible_moves = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i, j))
            ridx = np.random.choice(len(possible_moves))
            next_move = possible_moves[ridx]
        else:
            next_move = None
            best_value = -2
            pos2value = {}
            if self.verbose:
                if self.sym == env.x:
                    print('x', end='')
                elif self.sym == env.o:
                    print('o', end='')
                print(" is taking a greedy action - exploiting")
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        # Make move and get the state
                        env.board[i, j] = self.sym
                        state = env.get_state()
                        value = self.V[state]
                        pos2value[(i, j)] = value
                        # Unmake move
                        env.board[i, j] = 0
                        if value > best_value:
                            best_value = value
                            best_state = state
                            next_move = (i, j)
            if self.verbose:
                for i in range(LENGTH):
                    for j in range(LENGTH):
                        pchar = '     '
                        if env.is_empty(i, j):
                            pchar = '{:6.3f}'.format(pos2value[(i,j)])
                        else:
                            if env.board[i, j] == env.x:
                                pchar = '   x  '
                            elif env.board[i, j] == env.o:
                                pchar = '   o  '
                        if j < 2:                  
                            print(' {} |'.format(pchar), end="")
                        else:
                            print(' {} '.format(pchar))
                    if i < 2:
                        print('--------+--------+--------')
                    else:
                        print()                
        env.board[next_move[0], next_move[1]] = self.sym

    def update_state_history(self, s):
        self.state_history.append(s)
    
    def update(self, env):
        # At the end of an episode (complete game), update the values
        # Backtrack over the states so 
        # V(prev_state) = V(prev_state) + alpha * (V(next_state) - V(prev_state))
        # where V(next_state) = reward if it is the most current state
        reward = env.reward(self.sym)
        target = reward
        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha * (target - self.V[prev]) 
            if self.verbose_update:
                pchar = '?'
                if self.sym == env.x:
                    pchar = 'x'
                elif self.sym == env.o:
                    pchar = 'o'
                print('{} target = {}, old previous = {}, new previous = {}'.format(pchar, target, self.V[prev], value))
                env.draw_state(prev)
            self.V[prev] = value
            target = value
        self.reset_history()
        self.update_epsilon()
        

In [ ]:
class Human:
    
    def __init__(self):
        pass
    
    def set_symbol(self, _sym):
        self.sym = _sym
        
    def take_action(self, env):
        while True:
            # break if we make a legal move
            move = input("Enter coordinates i, j for your next move (i,j = 0, 1, 2)")
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i, j):
                env.board[i, j] = self.sym
                break
                
    def update(self, env):
        pass
    
    def update_state_history(self, s):
        pass

In [ ]:
def get_state_hash_and_winner(env, i=0, j=0):
    results = []
    for v in (0, env.x, env.o):
        env.board[i, j] = v
        if j == 2:
            if i == 2:
                # We are at the last square of the board, so we're ready to evaluate 
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.get_winner()
                results.append((state, winner, ended))
            else:
                results += get_state_hash_and_winner(env, i + 1, 0)
        else:
            results += get_state_hash_and_winner(env, i, j + 1)
    return results

def initialV_x(env, state_winner_triples):
    # if x wins, V(s) = 1
    # if x loses, V(s) = -1 [vs 0 in course]
    # if it is a draw, V(s) = 0 [vs 0.5 in course]
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.x:
                v = 1
            elif winner == env.o:
                v = -1
            else:
                v = 0
            V[state] = v
    return V

def initialV_o(env, state_winner_triples):
    # if o wins, V(s) = 1
    # if o loses, V(s) = -1 [vs 0 in course]
    # if it is a draw, V(s) = 0 [vs 0.5 in course]
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.o:
                v = 1
            elif winner == env.x:
                v = -1
            else:
                v = 0
            V[state] = v
    return V



In [ ]:
def play_game(p1, p2, env, draw=False, announce_winner=False):
    current_player = None
    while not env.game_over():
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
        if draw:
            if draw == 2 and current_player == p1:
                env.draw_board()
            if draw == 2 and current_player == p2:
                env.draw_board()
        current_player.take_action(env)
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)
    if draw:
        env.draw_board
    if announce_winner:
        if env.winner == env.x:
            print('x wins!')
        elif env.winner == env.o:
            print('o wins!')
        else:
            print('Neither wins')
    p1.update(env)
    p2.update(env)

def set_all_verbose(p1, p2, verbose):
    p1.set_verbose(verbose)
    p1.set_verbose_update(verbose)
    p2.set_verbose(verbose)
    p2.set_verbose_update(verbose)


In [ ]:
env = Environment()
state_winner_triples = get_state_hash_and_winner(env)

p1 = Agent()
p1.set_symbol(env.x)
Vx = initialV_x(env, state_winner_triples)
p1.setV(Vx)

p2 = Agent()
p2.set_symbol(env.o)
Vo = initialV_o(env, state_winner_triples)
p2.setV(Vo)

set_all_verbose(p1, p2, False)
T = 100
for t in range(T):
    play_game(p1, p2, Environment())

set_all_verbose(p1, p2, True)
for t in range(1):
    play_game(p1, p2, Environment())

In [ ]:
human = Human()
human.set_symbol(env.o)
while True:
    p1.set_verbose(True)
    p1.set_verbose_update(False)
    play_game(p1, human, Environment(), draw=2, announce_winner=True)
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
        break